# Backpropagation through Back substitution with a Backslash
Notebook per il seminario d'esame per il corso di Metodi di Approssimazione 

## Algebra degli operatori

Definiamo il tipo di dato custom che rappresenta la funzione lineare

In [1]:
using LinearAlgebra

struct Operator  # Linear Matrix Operators from Matrices to Matrices (and the operator adjoint)
    op
    adj
    sym
end

## Operators
ℒ(A::Matrix)  = Operator(X->A*X   , X->A'*X, "ℒ$(size(A))"  )   # left multiply by A (X → AX)
ℛ(A::Matrix)  = Operator(X->X*A   , X->X*A', "ℛ$(size(A))")     # right multiply by A (X → XA)
ℋ(A::Matrix)  = Operator(X->X.*A  , X->X.*A, "ℋ$(size(A))")    # Hadamard product (elementwise product)
ℐ()  =          Operator(X->X      ,    X->X,    "I")     # identity operator
𝒪()  =           Operator(X->zero(X) , X->zero(X),"𝒪")# zero operator

𝒪 (generic function with 1 method)

Dobbiamo anche fare *overloading* delle operazioni

In [2]:
import Base:  zero, show, adjoint, *, \, ∘, +, -
show(io::IO, M::Operator) = print(io, M.sym)  # pretty printing
zero(::Any) = 𝒪() # Let's make any undefined zero the 𝒪 operator

adjoint(A::Operator) = Operator(A.adj, A.op,  "("*A.sym*")'")
adjoint(B::Bidiagonal) = Bidiagonal(adjoint.(B.dv),adjoint.(B.ev),(B.uplo == 'U') ? :L : :U) # lower to upper

-(A::Operator) = Operator(X->-A.op(X), X->-A.adj(X),"-"*A.sym)
-(::typeof(𝒪()), X::Matrix) = -X  # 𝒪() - X should be -X
\(ℐ::typeof(ℐ()), A::Matrix) = A    # left division with ℐ() does nothing
*(A::Operator, X::Matrix) = A.op(X)
∘(A::Operator, B::Operator) = Operator(A.op ∘ B.op, B.adj ∘ A.adj, A.sym*"∘"*B.sym)
# We need [A;B]*C to somehow magically be [AC;BC]
*(M::Adjoint{Operator, Matrix{Operator}},v::Array) = M .* [v]
+(A::Array,x::Number)=A.+x

+ (generic function with 192 methods)

## Una semplice rete neurale

In [3]:
using OffsetArrays

# funzione di attivazione
h(x)  = tanh(x)
h′(x) = 1 - h(x)^2


# questa funzione, di fatto, effettua il forward pass
function neural_net(params,X₀;h=h,h′= h′)
    T = Matrix{Float64}
    N = length(params)
    X = OffsetArray(Vector{T}(undef,N+1),0:N)   
    Δ = Vector{T}(undef, N)
    X[0] = X₀
    W = first.(params)
    B = last.(params)
    
    for i=1:N         
          X[i] =  h.(W[i]*X[i-1] .+ B[i])
          Δ[i] =  h′.(W[i]*X[i-1] .+ B[i])        
    end 
    X,Δ
end

neural_net (generic function with 1 method)

Adesso possiamo costruire la rete

In [4]:
## Un po' di parametri che descrivono la rete...

n = [5,4,3,1]   # contiene [n₀...n_N], le dimensioni dei layer
k = 10          # batchsize
N = length(n)-1 # numero di layer (nascosti + output). Dev'essere positivo

init(sizes...) = 0.01randn(sizes...)    # utility function per inizializzare

init (generic function with 1 method)

In [5]:
## Un po' di parametri degni di questo nome

# Creiamo le matrici dei pesi e i vettori dei bias
Ws_and_bs =[ [init(n[i+1],n[i]) , init(n[i+1])]  for i=1:N] # The second part of the pair is a vector here

X₀ = init(n[1],k)         # batch di k pattern (aka i dati)
y  = init(n[end],k);      # y is what we will compare X_N against (aka l'etichetta)

𝓁(x,y)  = sum(abs2,x-y)/2   # loss (errore quadratico)
𝓁′(x,y) = x .- y;           # derivata della loss (w.r.t. output layer)

X, δ = neural_net(Ws_and_bs,X₀) # Inferenza (aka forward pass)
# X e δ hanno i valori per ogni layer (servono per fare il backward pass)

([[-0.0016417890745344186 -0.009017347211780685 … 0.0025201846228055233 -0.005976180522108683; -0.006524349957957305 0.006668125733105701 … -0.001309900513298213 -0.011214868952770391; … ; 0.009675829835573083 0.005185615908738407 … 0.00448176661371606 -0.008001887746487812; -0.0006842970624141817 -0.01434530203871462 … 0.01449309417043186 0.026235373996079652], [-0.0035950161387576906 -0.0037450840253618186 … -0.0038194347346536457 -0.0034255935748631556; 0.001479865885599661 0.0011574033454949788 … 0.0014576388765281478 0.0012374923073648122; 0.004662760313738267 0.004699608816944095 … 0.004364163082760891 0.00432361808896052; -0.001092758455435424 -0.0008530184413698413 … -0.0013226168609899868 -0.0011771650889709278], [-0.0010305072324071163 -0.0010310707137579532 … -0.0010389906484406792 -0.0010339513011064557; 0.012250733425353719 0.012248764345552322 … 0.01224982871517388 0.012251318577506318; -0.004634072369190367 -0.004626532936839199 … -0.0046365253698575385 -0.00463433714334

Adesso possiamo calcolare il gradiente della loss usando la nostra tecnica

In [7]:
## The diagonal matrix
#M = Diagonal([ [ℋ(δ[i]) ∘ ℛ(X[i-1])  ℋ(δ[i])] for i=1:N]) # OLD!
M = Diagonal([ [ℋ(δ[i]) ∘ ℛ(X[i-1])  ℋ(δ[i]) ∘ ℛ(ones(1,k))] for i=1:N])

## The lower triangular matrix (I-L)
ImL = Bidiagonal([ℐ() for i in 1:N], -[ℋ(δ[i]) ∘ ℒ(Ws_and_bs[i][1]) for i=2:N] , :L)

## derivata della loss (rispetto all'output layer)
g = [ fill(𝒪(),N-1) ; [𝓁′(X[N],y)] ]      

## Finalmente, il gradiente della loss, usando il backslash
∇J = M' * (ImL' \ g)

3-element Vector{Matrix{Matrix{Float64}}}:
 [[1.4088127013625462e-8 -6.147652480622261e-9 … 1.4816419312868407e-8 -2.5388932382734446e-8; 1.0140479781016571e-7 -4.4250211946458045e-8 … 1.0664677341123823e-7 -1.8274649895991573e-7; -5.097415279758743e-8 2.2243637685757134e-8 … -5.3609164969686924e-8 9.186308901637223e-8; -1.3074494685783658e-8 5.7053466945935386e-9 … -1.3750373031774129e-8 2.3562181295239357e-8]; [-2.472493766068718e-7; -1.779680410303707e-6; 8.946151060642498e-7; 2.2945975531540416e-7;;];;]
 [[5.1684687361671e-8 6.358938748516488e-9 -4.773460222810447e-8 1.630541229054533e-8; -2.8385966094942884e-7 -3.492418946061795e-8 2.621652361170077e-7 -8.955164381597129e-8; -4.0106900857663703e-7 -4.934481338688734e-8 3.704166924592903e-7 -1.2652868134864764e-7]; [-1.4484995052276062e-5; 7.955365450264909e-5; 0.00011240239326399518;;];;]
 [[-8.376357555014538e-6 9.761445816787112e-5 -3.6788071913389015e-5]; [0.007968857159263123;;];;]

In [8]:
for i=1:N
    print("∇J[$i]\n")
    print("\tTipo: $(typeof(∇J[i]))\t size: $(size(∇J[i]))\n")
    print("\t\t∇J[$i][1] ha dimensione $(size(∇J[i][1])), e n_$(i) x n_$(i-1) = $(n[i+1])x$(n[i]). Tipo $(typeof(∇J[i][1]))\n")
    print("\t\t∇J[$i][2] ha dimensione $(size(∇J[i][2])), e n_$(i) \t   = $(n[i+1]).   Tipo $(typeof(∇J[i][2]))\n")
    print("\n")
end

∇J[1]
	Tipo: Matrix{Matrix{Float64}}	 size: (2, 1)
		∇J[1][1] ha dimensione (4, 5), e n_1 x n_0 = 4x5. Tipo Matrix{Float64}
		∇J[1][2] ha dimensione (4, 1), e n_1 	   = 4.   Tipo Matrix{Float64}

∇J[2]
	Tipo: Matrix{Matrix{Float64}}	 size: (2, 1)
		∇J[2][1] ha dimensione (3, 4), e n_2 x n_1 = 3x4. Tipo Matrix{Float64}
		∇J[2][2] ha dimensione (3, 1), e n_2 	   = 3.   Tipo Matrix{Float64}

∇J[3]
	Tipo: Matrix{Matrix{Float64}}	 size: (2, 1)
		∇J[3][1] ha dimensione (1, 3), e n_3 x n_2 = 1x3. Tipo Matrix{Float64}
		∇J[3][2] ha dimensione (1, 1), e n_3 	   = 1.   Tipo Matrix{Float64}



### Check

Calcoliamo il gradiente usando il metodo delle differenze finite, per controllare il risultato

In [9]:
#∇Jfd is the gradient calculated with finite differences method

# creiamo uno zero della stessa dimensione del gradiente
∇Jfd = Ws_and_bs*0
ϵ = Ws_and_bs*0
𝜀 = .0001
for i=1:length(Ws_and_bs), wb=1:2
    for j=1:length(ϵ[i][wb])    
        ϵ[i][wb][j] = 𝜀
        # Derivata direzionale approssimata con differenze finite
        ∇Jfd[i][wb][j] = (𝓁(neural_net(Ws_and_bs+ϵ,X₀)[1][N],y) .- 𝓁(neural_net(Ws_and_bs-ϵ,X₀)[1][N],y))/2𝜀
        ϵ[i][wb][j] = .0
    end
end

In [10]:
for i=1:N
    print("∇Jfd[$i]\n")
    print("\tTipo: $(typeof(∇Jfd[i]))\t size: $(size(∇Jfd[i]))\n")
    print("\t\t∇Jfd[$i][1] ha dimensione $(size(∇Jfd[i][1])), e n_$(i) x n_$(i-1) = $(n[i+1])x$(n[i]). Tipo $(typeof(∇Jfd[i][1]))\n")
    print("\t\t∇Jfd[$i][2] ha dimensione $(size(∇Jfd[i][2]))  , e n_$(i) \t     = $(n[i+1]).   Tipo $(typeof(∇Jfd[i][2]))\n")
    print("\n")
end

∇Jfd[1]
	Tipo: Vector{Array{Float64}}	 size: (2,)
		∇Jfd[1][1] ha dimensione (4, 5), e n_1 x n_0 = 4x5. Tipo Matrix{Float64}
		∇Jfd[1][2] ha dimensione (4,)  , e n_1 	     = 4.   Tipo Vector{Float64}

∇Jfd[2]
	Tipo: Vector{Array{Float64}}	 size: (2,)
		∇Jfd[2][1] ha dimensione (3, 4), e n_2 x n_1 = 3x4. Tipo Matrix{Float64}
		∇Jfd[2][2] ha dimensione (3,)  , e n_2 	     = 3.   Tipo Vector{Float64}

∇Jfd[3]
	Tipo: Vector{Array{Float64}}	 size: (2,)
		∇Jfd[3][1] ha dimensione (1, 3), e n_3 x n_2 = 1x3. Tipo Matrix{Float64}
		∇Jfd[3][2] ha dimensione (1,)  , e n_3 	     = 1.   Tipo Vector{Float64}



Interessante notare un leggero mismatch di tipi tra `∇J` e `∇Jfd`, ma restano compatibili. 

In [11]:
# flatten(J) = vcat((x->x[:]).(vcat(J...))...)

function flatten(J) 
    # create a 2N x 1 matrix of matrices [W₁; B₁; ... Wₙ]
    stackedWandB = vcat(J...)       # this is a Matrix{Matrix{Float64}}
    # flatten out the matrices inside the stack
    unstackedStack = (x->x[:]).(stackedWandB)   # this is a Matrix{Vector{Float64}}

    # stack all the vectors that are in the `unstackedStack` matrix
    return vcat(unstackedStack...)
end

flatten (generic function with 1 method)

In [12]:
norm(flatten(∇J)-flatten(∇Jfd))

1.6298909507136707e-10

In [ ]:
norm(flatten(∇J)-flatten(∇Jfd)) / norm(flatten(∇J))

2.044840802928026e-8

## Errori, e problemi

### In generale, moltiplicazione tra matrici di operatori

---

### Moltiplicare matrici di operatori

E' possibile moltiplicare ($\equiv$ comporre) matrici di operatori, usando le regole a nostra disposizione? E eventualmente aggiungendone di nuove?

In [22]:
Mnuova = Diagonal([ℋ(δ[i]) ∘ ℛ(X[i-1]) for i=1:N])

3×3 Diagonal{Operator, Vector{Operator}}:
 ℋ(4, 10)∘ℛ(5, 10)  ⋅                  ⋅
 ⋅                  ℋ(3, 10)∘ℛ(4, 10)  ⋅
 ⋅                  ⋅                  ℋ(1, 10)∘ℛ(3, 10)

In [23]:
try
    ImL * Mnuova
catch e
    showerror(stdout, e)
end

3×3 Bidiagonal{Operator, Vector{Operator}}:
 I∘ℋ(4, 10)∘ℛ(5, 10)                  …  ⋅
 -ℋ(3, 10)∘ℒ(3, 4)∘ℋ(4, 10)∘ℛ(5, 10)     ⋅
 ⋅                                       I∘ℋ(1, 10)∘ℛ(3, 10)

Come glielo spieghiamo che quell'elemento lì in realtà è zero??

**AGGIUSTARE**: Dando le martellate giuste, qualcosa si può fare

In [24]:
*(A::Operator, B::Operator) = A ∘ B

* (generic function with 191 methods)

In [25]:
+(A::Operator, B::Operator) = Operator(X->(A.op(X) + B.op(X)), X->(A.adj(X) + B.adj(X)), A.sym*" + "*B.sym)

+ (generic function with 193 methods)

In [26]:
Matrix(ImL) * Matrix(Mnuova)

3×3 Matrix{Operator}:
 𝒪∘𝒪 + 𝒪∘𝒪 + I∘ℋ(4, 10)∘ℛ(5, 10) + 𝒪                  …  𝒪∘ℋ(1, 10)∘ℛ(3, 10) + 𝒪∘𝒪 + I∘𝒪 + 𝒪
 𝒪∘𝒪 + I∘𝒪 + -ℋ(3, 10)∘ℒ(3, 4)∘ℋ(4, 10)∘ℛ(5, 10) + 𝒪     𝒪∘ℋ(1, 10)∘ℛ(3, 10) + I∘𝒪 + -ℋ(3, 10)∘ℒ(3, 4)∘𝒪 + 𝒪
 I∘𝒪 + -ℋ(1, 10)∘ℒ(1, 3)∘𝒪 + 𝒪∘ℋ(4, 10)∘ℛ(5, 10) + 𝒪     I∘ℋ(1, 10)∘ℛ(3, 10) + -ℋ(1, 10)∘ℒ(1, 3)∘𝒪 + 𝒪∘𝒪 + 𝒪

In [27]:
ImL * M

DimensionMismatch: DimensionMismatch: incompatible dimensions for matrix multiplication: tried to multiply a matrix of size (3, 4) with a matrix of size (1, 2). The second dimension of the first matrix: 4, does not match the first dimension of the second matrix: 1.